#1. Chargement et Prétraitement des Données


In [4]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator


ModuleNotFoundError: No module named 'tensorflow'

# Décompression du dataset


In [3]:
from google.colab import files
uploaded = files.upload()

Saving glaucomaData.zip to glaucomaData.zip


# Décompression


In [4]:
with zipfile.ZipFile("glaucomaData.zip", 'r') as zip_ref:
    zip_ref.extractall("dataset")


# Définir les chemins des dossiers


In [5]:
base_dir = "dataset"
train_dir = os.path.join(base_dir, "train")  # Par exemple : dataset/train
test_dir = os.path.join(base_dir, "test")    # Par exemple : dataset/test

# Prétraitement des données


In [10]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Définir le répertoire contenant les images
# train_dir = "/content/drive/MyDrive/glaucomaData.zip"
train_dir = "dataset" # Change to the extracted dataset directory

# Paramètres
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

# Générateur de données avec augmentation et validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,       # Normalisation des pixels
    validation_split=0.2  # 20% des données pour la validation
)

# Générateur pour l'ensemble d'entraînement
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',  # Adapté pour une classification binaire
    subset='training'     # Sous-ensemble pour l'entraînement
)

# Générateur pour l'ensemble de validation
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'   # Sous-ensemble pour la validation
)

Found 397 images belonging to 4 classes.
Found 99 images belonging to 4 classes.


#2. Construction et Entraînement du Modèle


# Modèle CNN


In [11]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Compilation


In [12]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Entraînement


In [13]:
history = model.fit(train_generator, validation_data=val_generator, epochs=10)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.1873 - loss: -76.2547 - val_accuracy: 0.2020 - val_loss: -728.0934
Epoch 2/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.1860 - loss: -1976.8247 - val_accuracy: 0.2020 - val_loss: -7240.0630
Epoch 3/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.2095 - loss: -13867.0713 - val_accuracy: 0.2020 - val_loss: -37500.1797
Epoch 4/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.2107 - loss: -66709.8438 - val_accuracy: 0.2020 - val_loss: -137966.8125
Epoch 5/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.1935 - loss: -220748.4531 - val_accuracy: 0.2020 - val_loss: -397896.8438
Epoch 6/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accuracy: 0.1911 - loss: -598931.8125 - val_accuracy: 0.2020 - val_loss: -959305.0000
Epoch 7/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 16s 1s/step - accuracy: 0.1763 - loss: -1299283.6250 - val_accuracy: 0.2020 - val_loss: -1971500.2500
Epoch 8/10
13/13 ━━━━━━━━━━━━━━━━━━━━ 15s 1s/step - accur

# Sauvegarder le modèle

In [14]:
model.save("glaucoma_model.h5")
